In [ ]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
df = pd.read_csv("D:/nba_player_csv_current/season_2024-25/all_quarters/Alex Caruso_content.csv")
df


In [ ]:
import os
import pandas as pd
from pathlib import Path
from IPython.display import display

# Helper function to build the paths with customizable base path
def build_data_path(base_path, player, date):
    # Replace placeholders with actual values
    return base_path.format(date=date, player=player)
    

def his_player_defense_data(player, date, player_base_path, defense_base_path):
    player_dataframes = {}  # Dictionary to store each player's DataFrame
    defense_dataframes = {}  # Dictionary to store each player's defense DataFrame



    single_player_df = pd.DataFrame()  # Initialize a DataFrame for the current player
    defense_df = pd.DataFrame()  # Initialize a DataFrame for the current player's defense

    # Get the data paths using the helper function and the custom base paths
    path = build_data_path(player_base_path, player, date)
    #print(path)
    path_defense = build_data_path(defense_base_path, player, date)
    #print(path_defense)

    # Check if files exist and process them
    if os.path.exists(path) and os.path.exists(path_defense):
        # Player data
        season_df_player = pd.read_csv(path)
        season_df_player['season'] = date
        single_player_df = pd.concat([single_player_df, season_df_player], ignore_index=True)

        # Defense data
        defense_df_season = pd.read_csv(path_defense)
        defense_df_season['season_defense'] = date
        defense_df = pd.concat([defense_df, defense_df_season], ignore_index=True)
    else:
        print(f'{date} not found for {player} or defense')

    # Add player data to the dictionary
    player_dataframes[player] = single_player_df
    defense_dataframes[player] = defense_df

    # Merge the player and defense data on 'season' and 'TEAM' fields
    merged_df = pd.merge(single_player_df, defense_df, how='inner', left_on=['Away', 'season'], right_on=['TEAM', 'season_defense']).reset_index(drop=True)
    merged_df = merged_df.sort_values(by="Date")

    pd.set_option('display.max_rows', 1000)  # Maximum number of rows to display
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', 1000)  # Adjust column width for better readability

    

    return merged_df
# D:\nba_player_csv_historic\season_2022-23\all_quarters
# "D:\nba_player_csv_current\season_2024-25\all_quarters\Alex Caruso_content.csv"
# "D:\nba_defense_history_csv\defense_csv_2022-23\all_quarter_defense_content.csv"
# "D:\nba_player_csv_historic\season_2022-23\all_quarters\Alex Caruso_content.csv"

    # example use case:
player = "Alex Caruso"
date = "2022-23"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
usage_base_path = ""

df =his_player_defense_data(player, date, player_base_path, defense_base_path)
df.head(1)
    




In [ ]:
from data_functions import his_player_defense_data
import pandas as pd

player_names = {
    'Shai Gilgeous-Alexander': 'OKC',
    'Alex Caruso': 'OKC',
    'Isaiah Hartenstein': 'OKC'
}
date_list = ['2022-23', '2023-24']

# Define your base paths with placeholders for dynamic insertion
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"

# Create a dictionary to hold each player's DataFrame
player_dfs = {}

for player, team in player_names.items():
    # Create a list to hold DataFrames for each date for the current player
    player_frames = []
    
    for date in date_list:
        # Read usage data for the given season/date
        usage_data = pd.read_csv(f"D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv")
        
        # Generate merged data using your custom function
        merged_data = his_player_defense_data(player_base_path, defense_base_path, player, date)
        
        # Add the season column to usage_data
        usage_data['season'] = date 
        
        # Get the player's usage percentage from the usage data
        # (Assumes that there is exactly one matching row)
        player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
        merged_data['USG'] = player_usage
        
        # Exclude rows where the TEAM column matches the given team
        merged_data = merged_data[merged_data['TEAM'] != team]
        
        # Select only the desired columns
        # merged_data = merged_data[['season', 'MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE']]
        
        # Append the DataFrame for this date to the player's list
        player_frames.append(merged_data)
    
    # Combine all dates for the current player into one DataFrame
    player_dfs[player] = pd.concat(player_frames, ignore_index=True)

# player_frames

# Now you have separate DataFrames for each player in the `player_dfs` dictionary.
# You can display or work with them individually:

for player, df in player_dfs.items():
    print(f"\nData for {player}:")
    display(df)


In [ ]:
pip install xgboost

In [ ]:
from data_functions import his_player_defense_data, current_player_defense_data
import pandas as pd
import numpy as np

#"D:\nba_player_csv_current\season_2024-25\all_quarters\Alex Caruso_content.csv"
#"D:\nba_defense_csv_current\defense_csv_2024-25\all_quarter_defense_content.csv"
#"D:\nba_scheduled_csv\schedule_csv_2025\ATL_schedule_content.csv"

player_names = {'Shai Gilgeous-Alexander':'OKC', 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC'}
date_list = ['2024-25']
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_current/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_csv_current/defense_csv_{date}/all_quarter_defense_content.csv"
# usage_data = f"D:/nba_usage_csv_current/usage_csv_{date}/{date}_content.csv"


current_player_dic = {}

for player, team in player_names.items():
  current_player_frames = []

  for date in date_list:
    usage_data = pd.read_csv(f"D:/nba_usage_csv_current/usage_csv_{date}/{date}_content.csv")  

    # use current_player function to to merge datat
    merged_data, current_player_defense = current_player_defense_data(player_base_path,defense_base_path,schedule_base_path,player,date,team)

    # Add the season column to usage_data
    usage_data['season'] = date
    
    # Get the player's usage percentage from the usage data
    # (Assumes that there is exactly one matching row)
    player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
    merged_data['USG'] = player_usage


    # adding current player team pace
    team_stat = current_player_defense.loc[current_player_defense['TEAM'] == team, 'PACE'].values[0]
    merged_data["team_pace"] = team_stat

    # Exclude rows where the TEAM column matches the given team
    merged_data = merged_data[merged_data['TEAM'] != team]


    merged_data = merged_data[['season','Date', 'Matchup' ,'MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

    # Dropping duplicates
    merged_data = merged_data.drop_duplicates()
    
    # Append the DataFrame for this date to the player's list
    current_player_frames.append(merged_data)

  # Combine all dates for the current player into one DataFrame
  current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)

# specific_player = 'Shai Gilgeous-Alexander'
# for player, df in current_player_dic.items():
#   if player == specific_player:
#     print(f"\nData for {player}:")
#     display(df)


specific_player = 'Shai Gilgeous-Alexander'
for player, df in current_player_dic.items():
  if player ==specific_player:
    print(f"\nData for {player}:")
    df['FGA_rolling_3'] = df['FGA'].rolling(window=3).mean()

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score


    # Selecting features and target
    features = ['FGA_rolling_3', 'USG', 'MIN_x', 'DefRtg', 'PACE', 'team_pace']
    target = 'FGA'

    X = df[features]
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model training
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)

    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')


    print(f"Length of df['Matchup']: {len(df['Matchup'])}")
    print(f"Length of df['Date]: {len(df['Date'])}")
    print(f"Length of y_pred: {len(y_pred)}")
    print(f"Length of y_test: {len(y_test)}")
    matchups_test = df.loc[X_test.index, "Matchup"]
    date_test = df.loc[X_test.index, "Date"]
    print(f"length of df['Matchup']: {len(matchups_test)}")


    predicted_vs_actual = pd.DataFrame({
        "Date": date_test.values,  
        "Matchup": matchups_test.values,  
        "Predicted Points": y_pred.round(1), 
        "Actual Points": y_test}).reset_index(drop=True)
    
    print(predicted_vs_actual)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"Mean Squared Error: {mse}")
    print(f"RSME is: {rmse}")
    cv_rmse = np.sqrt(-cv_scores)

    # Print the RMSE for each fold
    print(f"Cross-Validation RMSE for each fold: {cv_rmse}")

    # Print the average RMSE across all folds
    print(f"Average Cross-Validation RMSE: {cv_rmse.mean()}")

    display(df.head(5))

In [ ]:
import pandas as pd
import os

# Define paths for the player, defense, and schedule CSVs
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"

# Define parameters
player = "Shai Gilgeous-Alexander"
date = "2023-24"
schedule_team = "OKC"  # Team for schedule filtering

# Call the function
merged_df_schedule, defense_current_player = current_player_defense_data(
    player_base_path, defense_base_path, schedule_base_path, player, date, schedule_team
)

# **Displaying the merged data**
print("Merged Data (Player + Defense + Schedule):")
print(merged_df_schedule.head())  # Show first few rows

# **Displaying the extracted defense data (TEAM, PACE)**
print("\nDefense Data (TEAM, PACE):")
print(defense_current_player)


In [ ]:
import pandas as pd
import numpy as np

def predict_fga(player_data, alpha=0.3, forecast_periods=5):
    """
    Predict future FGA (Field Goal Attempts) using EWMA
    
    Parameters:
    player_data: pandas Series of historical FGA data
    alpha: smoothing factor (default: 0.3)
    forecast_periods: number of future games to predict
    
    Returns:
    Dictionary containing:
    - historical_ewma: EWMA values for historical data
    - predictions: predicted FGA values
    - confidence_intervals: upper and lower bounds for predictions
    """
    # Calculate EWMA for historical data
    historical_ewma = player_data.ewm(alpha=alpha, adjust=False).mean()
    
    # Calculate volatility (standard deviation of percentage changes)
    volatility = player_data.pct_change().std()
    
    # Make predictions
    last_value = historical_ewma.iloc[-1]
    predictions = [last_value] * forecast_periods
    
    # Calculate confidence intervals (95%)
    z_score = 1.96  # 95% confidence interval
    std_error = volatility * np.sqrt(np.arange(1, forecast_periods + 1))
    upper_bound = [last_value * (1 + z_score * se) for se in std_error]
    lower_bound = [last_value * (1 - z_score * se) for se in std_error]
    
    return {
        'historical_ewma': historical_ewma,
        'predictions': predictions,
        'confidence_intervals': {
            'upper': upper_bound,
            'lower': lower_bound
        }
    }

# Example usage
if __name__ == "__main__":
    # Sample player FGA data for last 10 games
    player_fga = pd.Series([
        15, 18, 12, 16, 14, 19, 13, 17, 15, 16
    ], name='FGA')
    
    # Make predictions
    results = predict_fga(player_fga, alpha=0.3, forecast_periods=3)
    
    # Create analysis DataFrame
    analysis = pd.DataFrame({
        'Game': range(1, len(player_fga) + 1),
        'Actual_FGA': player_fga,
        'EWMA_FGA': results['historical_ewma']
    })
    
    # Add predictions
    prediction_df = pd.DataFrame({
        'Game': range(len(player_fga) + 1, len(player_fga) + 4),
        'Predicted_FGA': results['predictions'],
        'Upper_Bound': results['confidence_intervals']['upper'],
        'Lower_Bound': results['confidence_intervals']['lower']
    })

In [ ]:
display(prediction_df)

In [ ]:
from data_functions import his_player_defense_data, current_player_defense_data
import pandas as pd
import numpy as np

#"D:\nba_player_csv_current\season_2024-25\all_quarters\Alex Caruso_content.csv"
#"D:\nba_defense_csv_current\defense_csv_2024-25\all_quarter_defense_content.csv"
#"D:\nba_scheduled_csv\schedule_csv_2025\ATL_schedule_content.csv"

player_names = {'Shai Gilgeous-Alexander':'OKC', 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC'}
date_list = ['2022-23', "2023-24", "2024-25"]
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
# usage_data = f"D:/nba_usage_csv_current/usage_csv_{date}/{date}_content.csv"


current_player_dic = {}

for player, team in player_names.items():
  current_player_frames = []

  for date in date_list:
    usage_data = pd.read_csv(f"D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv")  

    # use current_player function to to merge datat
    merged_data, current_player_defense = his_player_defense_data(player_base_path,defense_base_path,player,date)

    # Add the season column to usage_data
    usage_data['season'] = date
    
    # Get the player's usage percentage from the usage data
    # (Assumes that there is exactly one matching row)
    player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
    merged_data['USG'] = player_usage


    # adding current player team pace
    team_stat = current_player_defense.loc[current_player_defense['TEAM'] == team, 'PACE'].values[0]
    merged_data["team_pace"] = team_stat

    # Exclude rows where the TEAM column matches the given team
    merged_data = merged_data[merged_data['TEAM'] != team]
    #display(merged_data.head(5))

    merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

    # Turn date into seconds
    merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
    merged_data = merged_data.sort_values(by="Date_in_Seconds")

    # Turn Home/Away game into 1 and 0
    merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
    # Dropping duplicates
    merged_data = merged_data.drop_duplicates()
    
    # Append the DataFrame for this date to the player's list
    current_player_frames.append(merged_data)

  # Combine all dates for the current player into one DataFrame
  current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)



specific_player = 'Shai Gilgeous-Alexander'
for player, df in current_player_dic.items():

    print(f"\nData for {player}:")
    df['FGA_rolling_3'] = df['FGA'].rolling(window=3).mean()


    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score

    alpha = 0.2

    df['EWMA_FGA'] = df['FGA'].ewm(span=(2/(1-alpha)-1), adjust=False).mean()

    df['EWMA_FGA_2'] = df['FGA'].ewm(span=(2/alpha - 1), adjust=False).mean()

    alpha = 0.2  # Example smoothing factor
    df['Exp_smooth'] = 0  # Initialize column

    for i in range(1, len(df)):
        df.loc[i, 'Exp_smooth'] = alpha * df.loc[i, 'FGA'] + (1 - alpha) * df.loc[i - 1, 'Exp_smooth']

    display(df.head(24))

    # # Selecting features and target
    # features = ['USG', 'DefRtg', 'PACE', 'team_pace', 'home_away']
    # target = 'FGA'

    # X = df[features]
    # y = df[target]

    # timestamp = int(pd.Timestamp('12/31/2024').timestamp())


    # train_data = df[df['Date_in_Seconds'] < timestamp]  # Training: First 4 seasons
    # test_data = df[df['Date_in_Seconds'] >= timestamp]  # Testing: Most recent season

    # # display(train_data)


    # X_train = train_data[features]
    # y_train = train_data[target]
    # X_test = test_data[features]
    # y_test = test_data[target]


    # # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # # Model training
    # model = RandomForestRegressor(n_estimators=100, random_state=42)
    # model.fit(X_train, y_train)


    # y_pred = model.predict(X_test)

    # cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')


    # print(f"Length of df['Matchup']: {len(df['Matchup'])}")
    # print(f"Length of df['Date]: {len(df['Date'])}")
    # print(f"Length of y_pred: {len(y_pred)}")
    # print(f"Length of y_test: {len(y_test)}")
    # matchups_test = df.loc[X_test.index, "Matchup"]
    # date_test = df.loc[X_test.index, "Date"]
    # print(f"length of df['Matchup']: {len(matchups_test)}")


    # predicted_vs_actual = pd.DataFrame({
    #     "Date": date_test.values,  
    #     "Matchup": matchups_test.values,  
    #     "Predicted Points": y_pred.round(1), 
    #     "Actual Points": y_test}).reset_index(drop=True)
    
    # print(predicted_vs_actual)

    # # Evaluate the model
    # mse = mean_squared_error(y_test, y_pred)
    # rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    # print(f"Mean Squared Error: {mse}")
    # print(f"RSME is: {rmse}")
    # cv_rmse = np.sqrt(-cv_scores)

    # # Print the RMSE for each fold
    # print(f"Cross-Validation RMSE for each fold: {cv_rmse}")

    # # Print the average RMSE across all folds
    # print(f"Average Cross-Validation RMSE: {cv_rmse.mean()}")
    # std_fga = np.std(df['FGA'])
    # print("This is the std:",std_fga)
    
    # latest_ewma_fga = df['EWMA_FGA'].iloc[-1]
    # #display(df)

    # # Create a DataFrame for the next game (fill with last known values)
    # next_game_features = pd.DataFrame({
    #     'USG': [df['USG'].iloc[-1]],  # Last known USG%
    #     'DefRtg': "116.2",  # Last known DefRtg
    #     'PACE': "99.99",  # Last known Pace
    #     'team_pace': [df['team_pace'].iloc[-1]],  # Last known team pace
    #     'home_away': [1]  # Assume it's an away game (or adjust accordingly)
    # })

    # display(next_game_features)


    # # Predict next game's FGA
    # predicted_fga_next_game = model.predict(next_game_features)[0]

    # print(f"Predicted FGA for Next Game: {predicted_fga_next_game:.2f}")
    # display(df.head(2))

In [ ]:
from data_functions import his_player_defense_data, current_player_defense_data
import pandas as pd
import numpy as np

#"D:\nba_player_csv_current\season_2024-25\all_quarters\Alex Caruso_content.csv"
#"D:\nba_defense_csv_current\defense_csv_2024-25\all_quarter_defense_content.csv"
#"D:\nba_scheduled_csv\schedule_csv_2025\ATL_schedule_content.csv"

player_names = {'Shai Gilgeous-Alexander':'OKC', 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC'}
date_list = ['2022-23', "2023-24", "2024-25"]
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
# usage_data = f"D:/nba_usage_csv_current/usage_csv_{date}/{date}_content.csv"


current_player_dic = {}

for player, team in player_names.items():
  current_player_frames = []

  for date in date_list:
    usage_data = pd.read_csv(f"D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv")  

    # use current_player function to to merge datat
    merged_data, current_player_defense = his_player_defense_data(player_base_path,defense_base_path,player,date)

    # Add the season column to usage_data
    usage_data['season'] = date
    
    # Get the player's usage percentage from the usage data
    # (Assumes that there is exactly one matching row)
    player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
    merged_data['USG'] = player_usage


    # adding current player team pace
    team_stat = current_player_defense.loc[current_player_defense['TEAM'] == team, 'PACE'].values[0]
    merged_data["team_pace"] = team_stat

    # Exclude rows where the TEAM column matches the given team
    merged_data = merged_data[merged_data['TEAM'] != team]
    #display(merged_data.head(5))

    merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

    # Turn date into seconds
    merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
    merged_data = merged_data.sort_values(by="Date_in_Seconds")

    # Turn Home/Away game into 1 and 0
    merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
    # Dropping duplicates
    merged_data = merged_data.drop_duplicates()
    
    # Append the DataFrame for this date to the player's list
    current_player_frames.append(merged_data)

  # Combine all dates for the current player into one DataFrame
  current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)

specific_player = 'Shai Gilgeous-Alexander'

for player, df in current_player_dic.items():
  if player == specific_player:
    df = df

    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import  LinearRegression
    from sklearn.metrics import mean_absolute_error, r2_score


    features = ['MIN_x','USG', 'DefRtg', 'PACE', 'team_pace', 'home_away']
    target = 'FGA'

    X = df[features]
    y = df[target]

    timestamp = int(pd.Timestamp('12/31/2024').timestamp())


    train_data = df[df['Date_in_Seconds'] < timestamp]  # Training: First 4 seasons
    test_data = df[df['Date_in_Seconds'] >= timestamp]  # Testing: Most recent season

    # display(train_data)


    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]


    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model training
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("R² Score:", r2_score(y_test, y_pred))
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("RMSE:", rmse)



In [ ]:
def build_data_path(base_path, **Kwargs):
    # Replace placeholders with actual values
    for key, value in Kwargs.items():
        base_path=base_path.replace(f"{{{key}}}", str(value))
    return base_path

base_path = "/data/{year}/{month}/{day}/file.txt"
kwargs = {'year': 2025, 'month': '02', 'day': '10'}
year = 2025
build_data_path(base_path, year=year)

In [ ]:
from data_functions import his_player_defense_data, current_player_defense_data, build_data_path
import pandas as pd
import numpy as np
from IPython.display import display

def fga_prediction_ewma(player_names: dict, date_list: list, usage_path,player_base_path,defense_base_path):
    current_player_dic = {}

    for player, team in player_names.items():
        current_player_frames =[]

        for date in date_list:
            usage_path =build_data_path(usage_path,date)
            usage_data = pd.read_csv(usage_path)

            #merging player and defense dat into one
            merged_data, current_defense_df = his_player_defense_data(player_base_path,defense_base_path,player,date)

            #adding season to usage_data
            usage_data['season'] = date

            #Getting the player usage percentage for usage data and adding to merge
            player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
            merged_data['USG'] = player_usage

            #adding the current player team pace
            team_stat = current_defense_df.loc[current_defense_df['TEAM'] == team, 'PACE'].values[0]
            merged_data["team_pace"] = team_stat
            
            # Exclude rows where the TEAM column matches the given team
            merged_data = merged_data[merged_data['TEAM'] != team]


            merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

            

            



            


In [1]:
from data_functions import his_player_defense_data, current_player_defense_data
import pandas as pd
import numpy as np

#"D:\nba_player_csv_current\season_2024-25\all_quarters\Alex Caruso_content.csv"
#"D:\nba_defense_csv_current\defense_csv_2024-25\all_quarter_defense_content.csv"
#"D:\nba_scheduled_csv\schedule_csv_2025\ATL_schedule_content.csv"

player_names = {'Shai Gilgeous-Alexander':'OKC', 'Alex Caruso':'OKC', 'Isaiah Hartenstein':'OKC'}
date_list = ["2022-23","2023-24","2024-25"]
schedule_base_path = "D:/nba_scheduled_csv/schedule_csv_2025/{schedule_team}_schedule_content.csv"
player_base_path = "D:/nba_player_csv_historic/season_{date}/all_quarters/{player}_content.csv"
defense_base_path = "D:/nba_defense_history_csv/defense_csv_{date}/all_quarter_defense_content.csv"
# usage_data = f"D:/nba_usage_csv_current/usage_csv_{date}/{date}_content.csv"



current_player_dic = {}

for player, team in player_names.items():
  current_player_frames = []

  for date in date_list:
    usage_data = pd.read_csv(f"D:/nba_usage_csv_historic/usage_csv_{date}/{date}_content.csv")  

    # use current_player function to to merge datat
    merged_data, current_player_defense = his_player_defense_data(player_base_path,defense_base_path,player,date)

    # Add the season column to usage_data
    usage_data['season'] = date
    
    # Get the player's usage percentage from the usage data
    # (Assumes that there is exactly one matching row)
    player_usage = usage_data.loc[usage_data['Player'] == player, 'USG%'].values[0]
    merged_data['USG'] = player_usage


    # adding current player team pace
    team_stat = current_player_defense.loc[current_player_defense['TEAM'] == team, 'PACE'].values[0]
    merged_data["team_pace"] = team_stat

    display(current_player_defense)

    # adding current player team OffRtg
    team_offrtg = current_player_defense.loc[current_player_defense['TEAM'] == team, 'OffRtg'].values[0]
    merged_data["team_offrtg"] = team_offrtg

    # Exclude rows where the TEAM column matches the given team
    merged_data = merged_data[merged_data['TEAM'] != team]
    #display(merged_data.head(5))

    # merged_data = merged_data[['season','Date', 'Home/Away_game' ,'Matchup' ,'PTS','MIN_x', 'Team', 'TEAM', 'FGA', 'USG', 'DefRtg', 'PACE','team_pace']]

    # Turn date into seconds
    merged_data['Date_in_Seconds'] = pd.to_datetime(merged_data['Date']).astype('int64') // 10**9
    merged_data = merged_data.sort_values(by="Date_in_Seconds")

    # Turn Home/Away game into 1 and 0
    merged_data['home_away'] = merged_data['Home/Away_game'].apply(lambda x: 1 if x == 'Away' else 0)
    # Dropping duplicates
    merged_data = merged_data.drop_duplicates()
    
    # Append the DataFrame for this date to the player's list
    current_player_frames.append(merged_data)

  # Combine all dates for the current player into one DataFrame
  current_player_dic[player] = pd.concat(current_player_frames, ignore_index=True)



specific_player = 'Shai Gilgeous-Alexander'
for player, df in current_player_dic.items():

    print(f"\nData for {player}:")
    df['FGA_rolling_3'] = df['FGA'].rolling(window=3).mean()



    alpha = 0.2

    df['EWMA_FGA'] = df['FGA'].ewm(span=(2/(1-alpha)-1), adjust=False).mean()

    df['EWMA_FGA_2'] = df['FGA'].ewm(span=(2/alpha - 1), adjust=False).mean()

    # alpha = 0.2  # Example smoothing factor
    df['Exp_smooth'] = 21  # Initialize column

    for i in range(1, len(df)):
        df.loc[i, 'Exp_smooth'] = alpha * df.loc[i, 'FGA'] + (1 - alpha) * df.loc[i - 1, 'Exp_smooth']

    df_act = df

    # display(df_act)

    df

    value_fga_list = []
    moving_average_list = []
    next_next_value_list = []
    date_list = []
    
    for value_fga, moving_average, date in zip(df['FGA'],df['EWMA_FGA_2'],df['Date']):
       value_fga_list.append(value_fga)
       moving_average_list.append(moving_average)
       #print("this is actual:",value_fga, "this is last_predicted",moving_average)
       next_next_value = alpha * value_fga + (1 - alpha) * moving_average
       next_next_value_list.append(next_next_value)
       date_list.append(date)
       #print("this is next:",next_next_value)
       dataframe_dic= {'Date':date_list,'Actual_FGA': value_fga_list, 'Moving_average':moving_average_list, 'Next':next_next_value_list}
       dataframe = pd.DataFrame(dataframe_dic)
       dataframe['Next'] = dataframe['Next'].shift(1)

    #display(dataframe)


    # Predict the next (11th) value
    last_actual = df['FGA'].iloc[-1]  # Last known FGA
    last_smoothed = df['EWMA_FGA_2'].iloc[-1]  # Last smoothed value
    next_value = alpha * last_actual + (1 - alpha) * last_smoothed


    # Print and add the prediction
    print(f"Predicted value: {next_value}")

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error



    features = ['EWMA_FGA_2', 'PACE', 'team_pace', 'USG', 'DefRtg','MIN_x', 'home_away', 'Date_in_Seconds','OffRtg', 'team_offrtg']
    target = 'FGA'


     # Continue with your existing operations
    timestamp = int(pd.Timestamp('2024-12-31').timestamp())
    train_data = df[df['Date_in_Seconds'] < timestamp]  # Replace '2023-01-01' with the corresponding timestamp
    test_data =  df[df['Date_in_Seconds'] >= timestamp]  # Replace '2023-01-01' with the corresponding timestamp


    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]


    model = LinearRegression()
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

     # Compare predictions with actual points
    predicted_vs_actual = pd.DataFrame({
        "Date": test_data['Date'],  
        "Matchup": test_data['Matchup'],  
        "Predicted Points": y_pred.round(1), 
        "Actual Points": y_test}).reset_index(drop=True)
    
    display(predicted_vs_actual)

    # X_future = pd.DataFrame({
    # 'MIN': result['Min'],         
    # 'FGA': result['predicted_fga'],         
    # 'FG%': result['predicted_fg_percentage'],         
    # 'FTA': result['predicted_fta'],          
    # 'DefRtg': result['DefRtg']    
    # })

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    print(f"MAE: {mae}, RMSE: {rmse}")

    import numpy as np

    # Drop NaN values (first row of 'Next' is NaN due to shift)
    rmse_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Compute RMSE
    rmse = np.sqrt(((rmse_df['Actual_FGA'] - rmse_df['Next']) ** 2).mean())

    print("RMSE:", rmse)


    # Drop NaN values (due to shift)
    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Total number of valid comparisons
    total = len(comparison_df)

    # Count occurrences
    higher_count = (comparison_df['Next'] > comparison_df['Actual_FGA']).sum()
    lower_count = (comparison_df['Next'] < comparison_df['Actual_FGA']).sum()
    equal_count = (comparison_df['Next'] == comparison_df['Actual_FGA']).sum()

    # Calculate percentages
    higher_percent = (higher_count / total) * 100
    lower_percent = (lower_count / total) * 100
    equal_percent = (equal_count / total) * 100

    # Print results
    print(f"Percentage of predictions HIGHER than actual: {higher_percent:.2f}%")
    print(f"Percentage of predictions LOWER than actual: {lower_percent:.2f}%")
    print(f"Percentage of predictions EQUAL to actual: {equal_percent:.2f}%")


    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Filter cases where Next is higher than Actual
    higher_cases = comparison_df[comparison_df['Next'] > comparison_df['Actual_FGA']]
    higher_difference_avg = (higher_cases['Next'] - higher_cases['Actual_FGA']).mean()

    # Filter cases where Next is lower than Actual
    lower_cases = comparison_df[comparison_df['Next'] < comparison_df['Actual_FGA']]
    lower_difference_avg = (lower_cases['Actual_FGA'] - lower_cases['Next']).mean()

    # Print results
    print(f"Average difference when prediction is HIGHER: {higher_difference_avg:.2f}")
    print(f"Average difference when prediction is LOWER: {lower_difference_avg:.2f}")

    # Drop NaN values
    comparison_df = dataframe.dropna(subset=['Actual_FGA', 'Next'])

    # Compute Mean Error (ME)
    mean_error = (comparison_df['Next'] - comparison_df['Actual_FGA']).mean()

    # Print result
    print(f"Mean Error (ME): {mean_error:.2f}")

    

D:/nba_player_csv_historic/season_2022-23/all_quarters/Shai Gilgeous-Alexander_content.csv
D:/nba_defense_history_csv/defense_csv_2022-23/all_quarter_defense_content.csv


,TEAM,PACE
0,MIL,101.45
1,BOS,99.15
2,PHI,97.44
3,DEN,98.74
4,CLE,96.27
5,MEM,101.50
6,SAC,100.99
7,NYK,97.75
8,BKN,98.77
9,PHX,98.83


KeyError: 'OffRtg'